# GRASS GIS intro

In this first part we will demonstrate starting GRASS GIS, creating new project and basic data visualization.
We will first use GUI and then we replicate it in the notebook environment running on mybinder.com.

## GUI intro

## GRASS GIS in a notebook

### Notebook environment

By default all cells are running Python:

In [ ]:
import sys
v = sys.version_info
print(f"We are using Python {v.major}.{v.minor}.{v.micro}")

However we will also show how to use GRASS from shell, for that we will use IPython magic:

In [ ]:
%%bash
pwd

### Start GRASS GIS

Create new empty location (project) that uses projection [UTM zone 17 N](https://epsg.io/6346).

In [ ]:
%%bash
grass -c EPSG:6346 -e ~/grassdata/dix_park/ 

Start GRASS GIS session in the newly created location. Load Python libraries.

In [ ]:
# Import Python standard library and IPython packages we need.
import os
import subprocess
import sys

# Ask GRASS GIS where its Python packages are.
gisbase = subprocess.check_output(["grass", "--config", "path"], universal_newlines=True).strip()
os.environ["GISBASE"] = gisbase
sys.path.append(os.path.join(gisbase, "etc", "python"))

# Import the GRASS GIS packages we need.
import grass.script as gs
import grass.jupyter as gj

# Start GRASS Session
gj.init("/home/anna/grassdata", "dix_park", "PERMANENT")

### Import data

Import prepared DSM, DTM and ortho. In this case no reprojection is done since the data CRS matches the location's CRS.

In [ ]:
%%bash
r.import input=dsm.tif output=dsm resample=bilinear
r.import input=ground.tif output=ground
r.import input=ortho.tif output=ortho

Download OSM data using Overpass Turbo.

We extract roads for our study area. The query should run upon running the cell below, so
it should be enough to *press Export - download as GeoJSON* and save it as `roads.geojson`.

In [ ]:
import IPython
IPython.display.IFrame('https://overpass-turbo.eu/s/1aGu', width=800, height=500)

Now we import the downloaded file. This time the data will be automatically reprojected because the CRS doesn't match.

In [ ]:
%%bash
v.import input=roads.geojson output=roads

## Raster buffer

Set computational region and create multiple buffers in given distances
around lakes represented as raster:

In [ ]:
gs.parse_command("g.region", raster="lakes", flags="pg")
gs.run_command("r.buffer", input="lakes", output="lakes_buff", distances=[60, 120, 240, 500])

# Start a GrassRenderer
img = gj.GrassRenderer()

# Add a raster and vector to the map
img.d_rast(map="lakes_buff")
img.d_legend(raster="lakes_buff", range=(2, 5), at=(80, 100, 2, 10))

# Display map
img.show()

## Vector buffer

Create a negative buffer around state boundary represented as a vector.
Vector modules typically don't follow computational region,
but we set it to inform display modules about our area of interest.

In [ ]:
gs.run_command("v.buffer", input="boundary_state", output="buffer", distance=-10000)
gs.parse_command("g.region", vector="boundary_state", flags="pg")

# Start another GrassRenderer
img2 = gj.GrassRenderer()

# Add vector layers and legend
img2.d_vect(map="boundary_state", fill_color="#5A91ED", legend_label="State boundary")
img2.d_vect(map="buffer", fill_color="#F8766D", legend_label="Inner portion")
img2.d_legend_vect(at=(10, 35))

# Display map
img2.show()

## Additional GRASS Information and Tutorials

To find more information on what one can do with GRASS GIS APIs, check out:
    
 - [GRASS GIS Manual](https://grass.osgeo.org/grass-stable/manuals)
    
 - [GRASS Python API Manual](https://grass.osgeo.org/grass-stable/manuals/libpython)

For more Jupyter Notebook GRASS GIS tutorials, visit:
 - [Try GRASS GIS online](https://grass.osgeo.org/learn/tryonline/)

## What else is in the sample North Carolina dataset?

In [ ]:
print(gs.read_command("g.list", type="all"))

## What other GRASS modules can I try in this notebooks?

In [ ]:
print(gs.read_command("g.search.modules", flags="g"))